In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# ===== Cấu hình Chrome =====
chrome_options = Options()
 # bỏ nếu muốn xem trình duyệt
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# ===== Danh sách URL muốn crawl =====
urls = {
    "Điện thoại": "https://www.lazada.vn/catalog/?q=điện%20thoại",
    "Quần áo": "https://www.lazada.vn/catalog/?q=quần%20áo",
    "Tủ lạnh": "https://www.lazada.vn/catalog/?q=tủ%20lạnh",
    "Tivi": "https://www.lazada.vn/catalog/?q=ti%20vi"
}

# ===== Hàm scroll để load thêm sản phẩm =====
def scroll_page():
    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(5):  # scroll 5 lần (có thể tăng nếu muốn nhiều hơn)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

# ===== Hàm crawl dữ liệu 1 URL =====
def crawl_lazada(url, category, max_items=30):
    driver.get(url)
    time.sleep(4)  # đợi trang load

    scroll_page()

    data = []
    products = driver.find_elements(By.CSS_SELECTOR, "div.RfADt")
    print(f"[{category}] tìm thấy {len(products)} sản phẩm")

    for p in products[:max_items]:
        try:
            name = p.text.strip()
        except:
            name = "N/A"

        try:
            price = p.find_element(By.XPATH, ".//following::span[@class='ooOxS'][1]").text
        except:
            price = "N/A"

        try:
            sold_count = p.find_element(By.XPATH, ".//following::span[contains(text(),'Đã bán')]").text
        except:
            sold_count = "N/A"

        data.append({
            "Category": category,
            "Name": name,
            "Price": price,
            "SoldCount": sold_count
        })

    return data

# ===== Crawl toàn bộ URL =====
all_data = []
for category, url in urls.items():
    print(f"Đang crawl {category} ...")
    results = crawl_lazada(url, category, max_items=30)
    all_data.extend(results)

# ===== Lưu ra Excel =====
df = pd.DataFrame(all_data)
df.to_excel("lazada_products.xlsx", index=False)
driver.quit()
print("✅ Crawl xong! Dữ liệu lưu vào lazada_products.xlsx")


Đang crawl Điện thoại ...
[Điện thoại] tìm thấy 40 sản phẩm
Đang crawl Quần áo ...
[Quần áo] tìm thấy 40 sản phẩm
Đang crawl Tủ lạnh ...
[Tủ lạnh] tìm thấy 40 sản phẩm
Đang crawl Tivi ...
[Tivi] tìm thấy 40 sản phẩm
✅ Crawl xong! Dữ liệu lưu vào lazada_products.xlsx
